In [ ]:
import cv2
import numpy as np
import threading
import socket
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

# --- Load YOLO ---
model = YOLO("model.pt")

# --- DeepSORT ---
tracker = DeepSort(max_age=150, max_iou_distance=0.4, nn_budget=100)

# --- Camera ---
ws, hs = 1280, 720
cap = cv2.VideoCapture(1)
cap.set(3, ws)
cap.set(4, hs)

# --- Socket TCP ---
HOST = '127.0.0.1'  # Địa chỉ IP của máy chủ (localhost)
PORT = 65432        # Cổng để gửi dữ liệu
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.connect((HOST, PORT))

def send_data(data):
    """Gửi dữ liệu qua socket."""
    serialized_data = data.tobytes()  # Chuyển đổi dữ liệu thành bytes
    server_socket.sendall(serialized_data)

# --- Dữ liệu chia sẻ giữa các luồng ---
detections = []
latest_frame = None
yolo_lock = threading.Lock()

# --- Hàm YOLO chạy song song ---
def run_yolo_loop():
    global detections, latest_frame
    while True:
        if latest_frame is not None:
            with yolo_lock:
                frame_copy = latest_frame.copy()
            results = model(frame_copy)
            new_detections = []
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = box.conf[0].item()
                class_id = int(box.cls[0].item())
                if class_id == 0:  # Người
                    new_detections.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_id))
            detections = new_detections

threading.Thread(target=run_yolo_loop, daemon=True).start()

# --- Mouse callback chọn người ---
cv2.namedWindow("Tracking")
selected_id = None

def select_object(event, x, y, flags, param):
    global selected_id
    if event == cv2.EVENT_LBUTTONDOWN:
        for track in param:
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            if x1 <= x <= x2 and y1 <= y <= y2:
                selected_id = track.track_id
                print(f"🔍 Chọn đối tượng ID: {selected_id}")

cv2.setMouseCallback("Tracking", select_object, param=[])

# --- Vòng lặp chính ---
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    with yolo_lock:
        latest_frame = frame.copy()

    tracks = tracker.update_tracks(detections, frame=frame)
    cv2.setMouseCallback("Tracking", select_object, param=tracks)

    # Hiển thị bounding box và nhận diện đối tượng
    for track in tracks:
        if track.is_confirmed():
            x1, y1, x2, y2 = map(int, track.to_ltrb())
            color = (0, 255, 0) if track.track_id == selected_id else (255, 0, 0)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"ID {track.track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Nếu có đối tượng được chọn, gửi đặc trưng của nó
    if selected_id is not None:
        selected_track = next((t for t in tracks if t.track_id == selected_id), None)
        if selected_track and selected_track.features:
            feature = selected_track.features[-1]  # Lấy đặc trưng cuối cùng
            send_data(feature)

    # Hiển thị khung hình
    cv2.imshow("Tracking", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
server_socket.close()